In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import keras
from keras.layers import Dense,Conv2D, BatchNormalization, Activation,Dropout
from keras.layers import AveragePooling2D, Input, Flatten,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
import keras.backend as K
import cv2
import seaborn as sns
import PIL
import skimage
from keras.models import Model
%matplotlib inline

Using TensorFlow backend.


In [3]:
def Preprocessing(x1, x2):
    x1_mean,x1_std = np.mean(x1, axis=(1,2,3)),np.std(x1, axis=(1,2,3))
    x2_mean,x2_std = np.mean(x2, axis=(1,2,3)),np.std(x2, axis=(1,2,3))
    x1 = (x1-x1_mean.reshape(-1,1,1,1))/x1_std.reshape(-1,1,1,1)
    x2 = (x2-x2_mean.reshape(-1,1,1,1))/x2_std.reshape(-1,1,1,1)
    return x1, x2

In [4]:
def data_generate(data_path, batch_size, argumentation = True, shuffle=False):
    while True:
        fid = h5py.File(data_path, 'r')
        data_len = fid['sen1'].shape[0]
        c = [i for i in range(int(data_len/batch_size) + 1)]
        if shuffle:
            np.random.shuffle(c)
        for i in c:
            t = np.random.rand()
            try:
                y_b = np.array((fid['label'][i * batch_size: (i+1)*batch_size]))
                x_b1 = fid['sen1'][i * batch_size: (i+1)*batch_size]
                x_b2 = fid['sen2'][i * batch_size: (i+1)*batch_size]
                #x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            #x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            #x_b3 = x_b3[:,:,::-1,:]
                x_b1, x_b2 = Preprocessing(x_b1, x_b2)
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                yield ([x_b1, x_b2, x_b3], y_b)
            except:
                x_b1 = fid['sen1'][i*batch_size:]
                x_b2 = fid['sen2'][i*batch_size:]
                
                y_b = np.array((fid['label'][i * batch_size:]))
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            #x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            #x_b3 = x_b3[:,:,::-1,:]
                x_b1, x_b2 = Preprocessing(x_b1, x_b2)
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                yield ([x_b1, x_b2, x_b3] , y_b)

In [5]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [7]:
def resnet_v2(input_tensor, depth, num_classes=17):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    #inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=input_tensor,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    #y = Flatten()(x)
    #outputs = Dense(num_classes,
     #               activation='softmax',
     #               kernel_initializer='he_normal')(y)

    # Instantiate model.
    #model = Model(inputs=inputs, outputs=outputs)
    return x

In [8]:
import keras.backend as K
import tensorflow as tf
def focal_loss(classes_num, gamma=2., alpha=.25, e=0.1):
    # classes_num contains sample number of each classes
    def focal_loss_fixed(target_tensor, prediction_tensor):
        '''
        prediction_tensor is the output tensor with shape [None, 100], where 100 is the number of classes
        target_tensor is the label tensor, same shape as predcition_tensor
        '''
        import tensorflow as tf
        from tensorflow.python.ops import array_ops
        from keras import backend as K

        #1# get focal loss with no balanced weight which presented in paper function (4)
        zeros = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        one_minus_p = array_ops.where(tf.greater(target_tensor,zeros), target_tensor - prediction_tensor, zeros)
        FT = -1 * (one_minus_p ** gamma) * tf.log(tf.clip_by_value(prediction_tensor, 1e-8, 1.0))

        #2# get balanced weight alpha
        classes_weight = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)

        total_num = float(sum(classes_num))
        classes_w_t1 = [ total_num / ff for ff in classes_num ]
        sum_ = sum(classes_w_t1)
        classes_w_t2 = [ ff/sum_ for ff in classes_w_t1 ]   #scale
        classes_w_tensor = tf.convert_to_tensor(classes_w_t2, dtype=prediction_tensor.dtype)
        classes_weight += classes_w_tensor

        alpha = array_ops.where(tf.greater(target_tensor, zeros), classes_weight, zeros)

        #3# get balanced focal loss
        balanced_fl = alpha * FT
        balanced_fl = tf.reduce_sum(balanced_fl)

        #4# add other op to prevent overfit
        # reference : https://spaces.ac.cn/archives/4493
        nb_classes = len(classes_num)
        fianal_loss = (1-e) * balanced_fl + e * K.categorical_crossentropy(K.ones_like(prediction_tensor)/nb_classes, prediction_tensor)

        return fianal_loss
    return focal_loss_fixed

In [9]:
input_shape1 = (32, 32, 8)
input_shape2 = (32, 32, 10)
input_shape3 = (32, 32, 18)
input_1 = Input(shape=input_shape1)
input_2 = Input(shape=input_shape2)
input_3 = Input(shape=input_shape3)
L1 = BatchNormalization()(input_1)
L2 = BatchNormalization()(input_2)
L3 = BatchNormalization()(input_3)
L1 = resnet_v2(input_tensor=L1, depth=92)
L2 = resnet_v2(input_tensor=L2, depth=92)
L3 = resnet_v2(input_tensor=L3, depth=92)

#L1 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L1)
#L2 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L2)


L = keras.layers.concatenate([L1, L2, L3])
L = Dense(512, activation='relu')(L)
L = BatchNormalization()(L)
L = Dropout(0.5)(L)
L = Dense(17, activation='softmax', kernel_initializer='he_normal')(L)
model = Model(inputs=[input_1, input_2, input_3], outputs=L)
class_nums = [5068, 24431, 31693,  8651, 16493, 35290,  3269, 39326,
       13584, 11954, 42902,  9514,  9165, 41377,  2392,  7898,
       49359]
#model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.compile(loss=focal_loss(class_nums), optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [10]:
save_dir = './models'
model_name = 'LCZ_model_3_inputsss-950-loss-f-92p.h5'
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',save_best_only=True, save_weights_only=True)
lr_reducr = ReduceLROnPlateau(factor=0.1,cooldown=0, patience=3, min_lr=1e-6)
early = EarlyStopping(monitor='val_acc',patience=5)
calls = [checkpoint, lr_reducr, early]

In [11]:
train_gen = data_generate('./training.h5', batch_size=128, argumentation=True, shuffle=True)
val_gen = data_generate('./validation.h5', batch_size=128, argumentation=True,shuffle=False)

In [12]:
model.fit_generator(train_gen, steps_per_epoch=352366//128+1, epochs=200,  verbose=1,callbacks=calls, validation_data=val_gen,
                   validation_steps=24119//128+1)

Epoch 1/200
2753/2753 [==============================] - 2553s 928ms/step - loss: 4.4713 - acc: 0.6953 - val_loss: 4.9647 - val_acc: 0.5769
Epoch 2/200
2753/2753 [==============================] - 2412s 876ms/step - loss: 2.2999 - acc: 0.7771 - val_loss: 4.9147 - val_acc: 0.5865
Epoch 3/200
2753/2753 [==============================] - 2401s 872ms/step - loss: 1.9101 - acc: 0.8056 - val_loss: 4.2664 - val_acc: 0.5788
Epoch 4/200
2753/2753 [==============================] - 2403s 873ms/step - loss: 1.7175 - acc: 0.8215 - val_loss: 5.1572 - val_acc: 0.5795
Epoch 5/200
2753/2753 [==============================] - 2416s 878ms/step - loss: 1.5906 - acc: 0.8346 - val_loss: 4.1035 - val_acc: 0.6012
Epoch 6/200
2753/2753 [==============================] - 2406s 874ms/step - loss: 1.5123 - acc: 0.8429 - val_loss: 4.5185 - val_acc: 0.6129
Epoch 7/200
2753/2753 [==============================] - 2393s 869ms/step - loss: 1.4460 - acc: 0.8514 - val_loss: 4.8029 - val_acc: 0.6047
Epoch 8/200
2753/275

In [11]:
def data_generate2(data1, data2, data3, batch_size, argumentation = True, shuffle=False):
    while True:
        data_len = data1.shape[0]
        c = [i for i in range(int(data_len/batch_size) + 1)]
        if shuffle:
            np.random.shuffle(c)
        for i in c:
            t = np.random.rand()
            try:
                y_b = data3[i*batch_size:(i+1)*batch_size]
                x_b1 = data1[i * batch_size: (i+1)*batch_size]
                x_b2 = data2[i * batch_size: (i+1)*batch_size]
                
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            #x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            #x_b3 = x_b3[:,:,::-1,:]
                x_b1, x_b2 = Preprocessing(x_b1, x_b2)
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                yield ([x_b1, x_b2, x_b3], y_b)
            except:
                y_b = data3[i*batch_size:]
                x_b1 = data1[i*batch_size:]
                x_b2 = data2[i*batch_size:]
                #x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            #x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            #x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            #x_b3 = x_b3[:,:,::-1,:]
                x_b1, x_b2 = Preprocessing(x_b1, x_b2)
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                yield ([x_b1, x_b2, x_b3], y_b)

In [12]:
valid = h5py.File('./validation.h5', 'r')
valid_s1 = valid['sen1']
valid_s2 = valid['sen2']
valid_label = valid['label']
valid_s1 = np.array(valid_s1)
valid_s2 = np.array(valid_s2)
valid_label = np.array(valid_label)

In [13]:
valid_s1.shape,valid_s2.shape,valid_label.shape

((24119, 32, 32, 8), (24119, 32, 32, 10), (24119, 17))

In [14]:
from sklearn.model_selection import StratifiedKFold, KFold
skf = StratifiedKFold(n_splits=5, random_state=2018)
kf = KFold(n_splits=5, random_state=2018)

In [15]:
y = np.argmax(valid_label, axis=1)

In [16]:
test = h5py.File('./round2_test_b_20190211.h5')
test_s1 = np.array(test['sen1'])
test_s2 = np.array(test['sen2'])

In [17]:
24119*0.8,24119*0.2

(19295.2, 4823.8)

In [18]:
pred = np.zeros((test_s1.shape[0], 17))
test_s3 = np.concatenate([test_s1, test_s2], axis=-1)
for i ,(tr_ind, val_ind) in enumerate(skf.split(valid_s1, y)):
    tr1, tr2, trl = valid_s1[tr_ind], valid_s2[tr_ind], valid_label[tr_ind]
    va1, va2, val = valid_s1[val_ind], valid_s2[val_ind], valid_label[val_ind]
    save_dir = './model_2/'
    model_name = 'LCZ_model_FOLD_3_input_%s_loss-f-92p.h5' % str(i)
    filepath = os.path.join(save_dir, model_name)
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',save_best_only=True, save_weights_only=True)
    lr_reducr = ReduceLROnPlateau(factor=0.5,cooldown=0, patience=2, min_lr=1e-6)
    early = EarlyStopping(monitor='val_acc',patience=4,)
    calls = [checkpoint, lr_reducr, early]
    class_num = [ 256., 1254., 2353.,  849.,  757., 1906.,  474., 3395., 1914.,
        860., 2287.,  382., 1202., 2747.,  202.,  672., 2609.]
    class_num = [int(x*0.8) for x in class_num]
    model.compile(loss=focal_loss(class_nums), optimizer=Adam(lr=0.0002), metrics=['accuracy'])
    model.load_weights('./models/LCZ_model_3_inputsss-950-loss-f-92p.h5')
    tr_gen = data_generate2(tr1, tr2, trl,argumentation=True,batch_size=128,shuffle=True)
    va_gen = data_generate2(va1, va2, val,argumentation=True,batch_size=128,shuffle=False)
    model.fit_generator(tr_gen, steps_per_epoch=19295//128+1, epochs=200, validation_data=va_gen,validation_steps=4823//128+1, callbacks=calls)
    model.load_weights(filepath)
    for j in range(4):
        te1 = np.rot90(test_s1,k=j,axes=(1,2))
        te2 = np.rot90(test_s2,k=j,axes=(1,2))
        te1, te2 = Preprocessing(te1, te2)
        te3 = np.concatenate([te1, te2], axis=-1)
        #te3 = np.rot90(test_s3,k=i,axes=(1,2))
        pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,::-1,:,:]
    te2 = test_s2[:,::-1,:,:]
    te1, te2 = Preprocessing(te1, te2)
    te3 = np.concatenate([te1, te2], axis=-1)
    pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,:,::-1,:]
    te2 = test_s2[:,:,::-1,:]
    te1, te2 = Preprocessing(te1, te2)
    te3 = np.concatenate([te1, te2], axis=-1)
    pred += model.predict([te1, te2, te3])

Epoch 1/200
151/151 [==============================] - 242s 2s/step - loss: 2.2930 - acc: 0.7578 - val_loss: 1.7508 - val_acc: 0.8189
Epoch 2/200
151/151 [==============================] - 128s 846ms/step - loss: 1.6879 - acc: 0.8188 - val_loss: 1.4696 - val_acc: 0.8561
Epoch 3/200
151/151 [==============================] - 127s 844ms/step - loss: 1.5103 - acc: 0.8407 - val_loss: 1.3476 - val_acc: 0.8640
Epoch 4/200
151/151 [==============================] - 128s 848ms/step - loss: 1.3970 - acc: 0.8557 - val_loss: 1.2772 - val_acc: 0.8853
Epoch 5/200
151/151 [==============================] - 126s 833ms/step - loss: 1.3152 - acc: 0.8711 - val_loss: 1.2205 - val_acc: 0.8841
Epoch 6/200
151/151 [==============================] - 128s 845ms/step - loss: 1.2529 - acc: 0.8811 - val_loss: 1.2358 - val_acc: 0.8864
Epoch 7/200
151/151 [==============================] - 128s 848ms/step - loss: 1.2098 - acc: 0.8886 - val_loss: 1.1802 - val_acc: 0.8957
Epoch 8/200
151/151 [=======================

Exception ignored in: <generator object data_generate2 at 0x7f51f823c360>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f5235871af0>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 142s 939ms/step - loss: 2.3532 - acc: 0.7534 - val_loss: 1.6718 - val_acc: 0.8095
Epoch 2/200
151/151 [==============================] - 123s 816ms/step - loss: 1.7004 - acc: 0.8169 - val_loss: 1.4470 - val_acc: 0.8379
Epoch 3/200
151/151 [==============================] - 123s 816ms/step - loss: 1.5181 - acc: 0.8382 - val_loss: 1.3370 - val_acc: 0.8548
Epoch 4/200
151/151 [==============================] - 121s 801ms/step - loss: 1.4061 - acc: 0.8542 - val_loss: 1.2952 - val_acc: 0.8528
Epoch 5/200
151/151 [==============================] - 123s 816ms/step - loss: 1.3276 - acc: 0.8655 - val_loss: 1.2555 - val_acc: 0.8805
Epoch 6/200
151/151 [==============================] - 124s 820ms/step - loss: 1.2798 - acc: 0.8751 - val_loss: 1.1807 - val_acc: 0.8847
Epoch 7/200
151/151 [==============================] - 123s 815ms/step - loss: 1.2118 - acc: 0.8850 - val_loss: 1.1605 - val_acc: 0.8927
Epoch 8/200
151/151 [====================

Exception ignored in: <generator object data_generate2 at 0x7f52358ae200>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f51f823c360>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 146s 966ms/step - loss: 2.2883 - acc: 0.7549 - val_loss: 1.7053 - val_acc: 0.8181
Epoch 2/200
151/151 [==============================] - 124s 818ms/step - loss: 1.6830 - acc: 0.8131 - val_loss: 1.4541 - val_acc: 0.8557
Epoch 3/200
151/151 [==============================] - 124s 822ms/step - loss: 1.4955 - acc: 0.8428 - val_loss: 1.3499 - val_acc: 0.8617
Epoch 4/200
151/151 [==============================] - 124s 819ms/step - loss: 1.3797 - acc: 0.8561 - val_loss: 1.2718 - val_acc: 0.8837
Epoch 5/200
151/151 [==============================] - 124s 820ms/step - loss: 1.3115 - acc: 0.8687 - val_loss: 1.2266 - val_acc: 0.8901
Epoch 6/200
151/151 [==============================] - 124s 819ms/step - loss: 1.2564 - acc: 0.8783 - val_loss: 1.2040 - val_acc: 0.8963
Epoch 7/200
151/151 [==============================] - 122s 809ms/step - loss: 1.2093 - acc: 0.8865 - val_loss: 1.1784 - val_acc: 0.8946
Epoch 8/200
151/151 [====================

Exception ignored in: <generator object data_generate2 at 0x7f4f7e238678>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f52358ae200>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 151s 1s/step - loss: 2.3229 - acc: 0.7538 - val_loss: 1.7222 - val_acc: 0.8108
Epoch 2/200
151/151 [==============================] - 124s 822ms/step - loss: 1.7079 - acc: 0.8137 - val_loss: 1.4395 - val_acc: 0.8486
Epoch 3/200
151/151 [==============================] - 124s 823ms/step - loss: 1.5183 - acc: 0.8369 - val_loss: 1.3332 - val_acc: 0.8637
Epoch 4/200
151/151 [==============================] - 125s 827ms/step - loss: 1.3953 - acc: 0.8543 - val_loss: 1.2578 - val_acc: 0.8809
Epoch 5/200
151/151 [==============================] - 121s 803ms/step - loss: 1.3169 - acc: 0.8690 - val_loss: 1.2558 - val_acc: 0.8728
Epoch 6/200
151/151 [==============================] - 124s 823ms/step - loss: 1.2804 - acc: 0.8781 - val_loss: 1.2132 - val_acc: 0.8849
Epoch 7/200
151/151 [==============================] - 124s 821ms/step - loss: 1.2125 - acc: 0.8876 - val_loss: 1.2091 - val_acc: 0.8959
Epoch 8/200
151/151 [=======================

Exception ignored in: <generator object data_generate2 at 0x7f4ff1e65f10>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f4f7e238678>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 154s 1s/step - loss: 2.3046 - acc: 0.7586 - val_loss: 1.6595 - val_acc: 0.8167
Epoch 2/200
151/151 [==============================] - 124s 824ms/step - loss: 1.6583 - acc: 0.8205 - val_loss: 1.4492 - val_acc: 0.8455
Epoch 3/200
151/151 [==============================] - 124s 822ms/step - loss: 1.4750 - acc: 0.8439 - val_loss: 1.3400 - val_acc: 0.8553
Epoch 4/200
151/151 [==============================] - 124s 822ms/step - loss: 1.3692 - acc: 0.8591 - val_loss: 1.2913 - val_acc: 0.8717
Epoch 5/200
151/151 [==============================] - 125s 825ms/step - loss: 1.3136 - acc: 0.8703 - val_loss: 1.2329 - val_acc: 0.8850
Epoch 6/200
151/151 [==============================] - 124s 824ms/step - loss: 1.2485 - acc: 0.8836 - val_loss: 1.2203 - val_acc: 0.8891
Epoch 7/200
151/151 [==============================] - 124s 824ms/step - loss: 1.1934 - acc: 0.8917 - val_loss: 1.2203 - val_acc: 0.9049
Epoch 8/200
151/151 [=======================

In [20]:
pred.shape

(4842, 17)

In [21]:
preds = pred / 30

In [22]:
np.sum(preds, axis=1)

array([1.        , 1.        , 1.        , ..., 1.00000001, 1.        ,
       1.00000001])

In [23]:
result = np.zeros((4842, 17))

In [24]:
for i ,ind in enumerate(np.argmax(preds,axis=1)):
    result[i][ind] = 1

In [25]:
result

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [26]:
import pandas as pd
s = pd.DataFrame(result,dtype='int')

In [27]:
s.to_csv('round2_74-result.csv', header=None, index=False)